Delta Task 2 Project WIP

In [54]:
!pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=b7fb4ba71c3edd433919aa9d2f6eb956594f5ed31ef2d6a31f14372e04eb03fe
  Stored in directory: /root/.cache/pip/wheels/e6/66/9c/712044a983337f5d44f90abcd244bd4b8ad28ee64750404b50
Successfully built wordninja


In [55]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import wordninja

In [29]:
#Reading train and test sets into data frame

btc_tweets_train = pd.read_parquet('btc_tweets_train.parquet.gzip')
btc_tweets_test = pd.read_parquet('btc_tweets_test.parquet.gzip')

First start with the exploratory data analysis to understand the data structure. Both training and test data has 5 columns and 1500 observations in train and 500 in test set.

In [26]:
btc_tweets_train.info()
btc_tweets_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 1641579121972236290 to 1641861677149822976
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   hashtags          1500 non-null   object
 1   content           1500 non-null   object
 2   username          1500 non-null   object
 3   user_displayname  1500 non-null   object
 4   sentiment         1500 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 60.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1641861708246552576 to 1641953216999968769
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   hashtags          500 non-null    object
 1   content           500 non-null    object
 2   username          500 non-null    object
 3   user_displayname  500 non-null    object
 4   sentiment         500 non-null    bool  
dtypes: bool(1), object(4)
memory usage

Content column has the content for sentiment analysis. The tweets include hashtags, emojis and line break elements, which may require handling before we start on tokenizing our data. And it is a mix of uppercase and lower case values

In [32]:
btc_tweets_train['content'][1641861088483368964]

'Last chance below 30k #BITCOIN\u202f\u202f\u202f #LFG \nSet a reminder for my upcoming Space! https://t.co/rnE6XDLOPH'

The data frame has an index column and the indexes for observations are the tweet_ID

In [23]:
btc_tweets_train.head(10)

,hashtags,content,username,user_displayname,sentiment
tweet ID,,,,,
1641579121972236290,"[Bitcoin, Bitcoin, BTC, Bitcoin, BTC, SHIB, HO...","$Bitcoin TO $100,000 SOONER THAN YOU THINK‼️💯🙏...",BezosCrypto,SHIB Bezos,True
1641579176171016194,"[Bitcoin, bitcoinordinals, crypto]",Alright I have my rares. Who else is grabbing ...,spartantc81,SpartanTC,True
1641579486071390208,"[BTC, SHIB, HOGE, SAITAMA, BNB, DOGE, ETH, Bab...","Bitcoin (BTC) Targets Over $100,000 as This Im...",BezosCrypto,SHIB Bezos,True
1641579537103302656,[BTC],📢 Xverse Web-based pool is live:\n\n•Update @x...,godfred_xcuz,Algorithm.btc,True
1641579588399804418,[Bitcoin],"Yesterday, a Bitcoin projection was displayed ...",goddess81oo,she is lucky,True
1641579652681543680,"[Crypto, Bitcoin, Investing]",Unpopular opinion:\n\nThis pump isn’t going to...,CloseSomeSayles,Andrew Sayles☕,False
1641579895615569921,[Bitcoin],#Bitcoin fixes this,ShannenJPEG,Shannen 🍉,True
1641580545678393345,[Bitcoin],Solid bid in major ALT/BTC pairs today. \n\nIf...,tedtalksmacro,tedtalksmacro,True
1641580600606769152,[bitcoin],"If you're filing 2022 taxes in the U.S., you s...",unchainedcom,Unchained,True


The sentiment of the content is already a boolean so dont need to change that. The train set consists heavily of positive sentiment score

In [24]:
btc_tweets_train.describe()

,hashtags,content,username,user_displayname,sentiment
count,1500,1500,1500,1500,1500
unique,945,1500,1012,1012,2
top,[Bitcoin],If the government are allowed to sell all the ...,BezosCrypto,SHIB Bezos,True
freq,384,1,41,41,1220


In [49]:
textual_sample = btc_tweets_train.iloc[0,1]
textual_sample

'$Bitcoin TO $100,000 SOONER THAN YOU THINK‼️💯🙏\n\n#Bitcoin\xa0TO $100,000 WHETHER YOU BELIEVE OR NOT‼️💯🙏\n\n$BTC #Bitcoin\xa0#BTC\xa0\xa0\xa0\n\n#Bitcoin\xa0#BTC\xa0#SHIB \n#HOGE #SAITAMA #BNB\xa0\xa0\xa0#DOGE #ETH #BabyFloki #AltCoinSeason https://t.co/rtlFlKlVCv'

In [50]:
#Removing of URLs:
preprocessed_tweets_train = re.sub("http\S+"," ",textual_sample)

print(preprocessed_tweets_train)

$Bitcoin TO $100,000 SOONER THAN YOU THINK‼️💯🙏

#Bitcoin TO $100,000 WHETHER YOU BELIEVE OR NOT‼️💯🙏

$BTC #Bitcoin #BTC   

#Bitcoin #BTC #SHIB 
#HOGE #SAITAMA #BNB   #DOGE #ETH #BabyFloki #AltCoinSeason  


In [51]:
#Changing all tokens to lower case
preprocessed_tweets_train = preprocessed_tweets_train.lower()
print(preprocessed_tweets_train)

$bitcoin to $100,000 sooner than you think‼️💯🙏

#bitcoin to $100,000 whether you believe or not‼️💯🙏

$btc #bitcoin #btc   

#bitcoin #btc #shib 
#hoge #saitama #bnb   #doge #eth #babyfloki #altcoinseason  


In [52]:
#Removing the line breaks:
preprocessed_tweets_train = preprocessed_tweets_train.replace('\n'," ")
print(preprocessed_tweets_train)

$bitcoin to $100,000 sooner than you think‼️💯🙏  #bitcoin to $100,000 whether you believe or not‼️💯🙏  $btc #bitcoin #btc     #bitcoin #btc #shib  #hoge #saitama #bnb   #doge #eth #babyfloki #altcoinseason  


In [58]:
hashtags = re.findall(r'#\w+', preprocessed_tweets_train)
for tag in hashtags:
  words= wordninja.split(tag[1:])
  preprocessed_tweets_train = preprocessed_tweets_train.replace(tag, ' '.join(words))

print(preprocessed_tweets_train)

$bitcoin to $100,000 sooner than you think‼️💯🙏  bitcoin to $100,000 whether you believe or not‼️💯🙏  $btc bitcoin btc     bitcoin btc shi b  hog e saitama b nb   doge eth baby f loki alt coin season  
